# User-Based Collaborative Filtering - Pearson Correlation

## 1. Set-up
import dependent packages and declare consts

In [1]:
# package initialization

import pandas as pd
import numpy as np
from surprise import BaselineOnly, KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn import metrics

In [18]:
# consts

# DATASET = '../../data/extracted/LON-A/London_Attractions_Complete_Review.csv'
DATASET = '../../data/extracted/NYC-R/New_York_City_Restaurant_Complete_Review.csv'
OCCURENCE_THRESHOLD = 5

## 2. Read Dataset
read dataset in csv format into pandas.DataFrame

In [19]:
# read dataset

df = pd.read_csv(DATASET, sep='\t')

In [20]:
# print dataset information

print("Columns: \n", list(df.columns))
print("\nShape: \n", df.shape)

Columns: 
 ['Unnamed: 0', 'Unnamed: 0.1', 'rtime', 'rquote', 'iid', 'rrate', 'rid', 'uage', 'ucity', 'ucountry', 'ugender', 'uhometown', 'uid_index', 'ulevel', 'uname_y', 'usince', 'ustate', 'ustyle', 'iattribute', 'ilocality', 'iname', 'ipopularity', 'ipost', 'iprice', 'irating', 'iregion', 'istreet', 'itag']

Shape: 
 (129964, 28)


## 3. Data Preprocessing

* Retain users/items with at least five ratings only
* Data splitting
  - the latest 20% interactions (by time)
  - Randomly split the remaining data into training (70%) and validation (10%) sets
* Transform the ratings into binary implicit feedback as ground truth, indicating whether the user has interacted with the specific item

In [5]:
def sort_by_time(df):
    
    # here we use 'rid' for sorting becaz it's auto incrementing
    return df.sort_values(by=['rid'], ascending=True)

In [6]:
def filter_by_occurrence(df, column, threshold):
    return df.groupby(column).filter(lambda x: len(x) >= threshold)

In [7]:
def convert_binary(df):
    df.loc[df['rrate'] != "None", 'rrate'] = 1.0
    df.loc[df['rrate'] == "None", 'rrate'] = 0.0
    return df

In [8]:
def data_preprocess(dataframe):
    
    # sort by time (ascending order)
    df = sort_by_time(dataframe)
    
    # retrieve needed columns
    df = df[['uid_index', 'iid', 'rrate']]
    
    # convert ratings into binarys
    df = convert_binary(df)
    
    df['rrate'] = pd.to_numeric(df['rrate'])
    
    # Retain users/items with at least five ratings only
    df = filter_by_occurrence(df, 'iid', OCCURENCE_THRESHOLD)
    df = filter_by_occurrence(df, 'uid_index', OCCURENCE_THRESHOLD)
    
    # split dataset into training set, validation set and test set
    users = df.groupby('uid_index')
    
    test_df = pd.DataFrame()
    train_validation_df = pd.DataFrame()
    
    # for each user, get its latest 20% rating as test set
    for uid in users.size().to_dict().keys():
        user = users.get_group(uid)
        split_idx = int(len(user)*0.8)
        test_df = test_df.append(user.iloc[split_idx:])
        train_validation_df = train_validation_df.append(user.iloc[:split_idx])
    
    train_validation_df = train_validation_df.reindex(np.random.permutation(train_validation_df.index)) # shuffle
    train_df = train_validation_df.iloc[:int(len(train_validation_df)*0.875)]
    validation_df = train_validation_df.iloc[int(len(train_validation_df)*0.875):]
    
    return (train_df, validation_df, test_df)

In [21]:
# dataset preprocessing

train_df, validation_df, test_df = data_preprocess(df)
print("training set size: ", train_df.shape)
print("validation set size: ", validation_df.shape)
print("test set size: ", test_df.shape)

training set size:  (80722, 3)
validation set size:  (11532, 3)
test set size:  (29517, 3)


## 4. Load into Surprise

In [22]:
reader = Reader(rating_scale=(0, 1))
train_dataset = Dataset.load_from_df(train_df, reader).build_full_trainset()

## 5. Model declaration & Fitting

Collaborative Filtering - KNN:

\begin{equation}
\hat{r}_{ui} = \frac{
\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}
{\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}
\end{equation}

**Parameters**
- `k` – The max number of neighbors to take into account for aggregation, defaults to `40`

In [11]:
# compute pearson correlation between users
sim_options = {'name': 'pearson', 'user_based': True}

K_MAX = 40

In [12]:
algorithm = KNNBasic(k=K_MAX, sim_options=sim_options)

In [13]:
algorithm.fit(train_dataset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


## 6. Prediction

In [11]:
def predict(algorithm, dataframe):
    z = []
    y = []
    for i in range(len(dataframe)):
        user = dataframe.iloc[i][0]
        item = dataframe.iloc[i][1]
        rating = dataframe.iloc[i][2]
        prediction = algorithm.predict(user, item, r_ui=rating, verbose=False)
        
        if prediction.details['was_impossible'] == False:
            z.append(prediction.est)
            y.append(rating)
    return (np.array(z, dtype=np.float32), np.array(y, dtype=np.int))

In [15]:
validation_z, validation_y = predict(algorithm, validation_df)

In [16]:
test_z, test_y = predict(algorithm, test_df)

## 7. Evaluation

AUC metric

In [12]:
def evaluate_auc(z, y):
    return metrics.roc_auc_score(y, z)

In [18]:
print("validation AUC: ", evaluate_auc(validation_z, validation_y))
print("test AUC: ", evaluate_auc(test_z, test_y))

validation AUC:  0.9979704105160415
test AUC:  0.995038099744845


LogLoss metric

In [13]:
# assume parameters z & y are ndarray
def evaluate_logloss(z, y):
    zz = np.ones((z.shape[0], 2))
    zz[:, 0] -= z
    zz[:, 1] = z
    return metrics.log_loss(y, zz)

In [20]:
print("validation LogLoss: ", evaluate_logloss(validation_z, validation_y))
print("test LogLoss: ", evaluate_logloss(test_z, test_y))

validation LogLoss:  0.051278227157755006
test LogLoss:  0.12634388294128107


NDCG metric

In [14]:
# assume parameters z & y are ndarray
def evaluate_ndcg(z, y):
    return metrics.ndcg_score(np.expand_dims(y, axis=0), np.expand_dims(z, axis=0), k=5)

In [22]:
print("validation NDCG@5: ", evaluate_ndcg(validation_z, validation_y))
print("test NDCG@5: ", evaluate_ndcg(test_z, test_y))

validation NDCG@5:  0.9994058229352346
test NDCG@5:  0.9987100935182198


## Experiments

In [15]:
def train_ks(start=3, end=20, step=1):

    history = []
    for k in range(start, end+step, step):
        
        print("Using K =", k)
        sim_options = {'name': 'cosine', 'user_based': False}
        algorithm = KNNBasic(k=k, sim_options=sim_options)
        
        algorithm.fit(train_dataset)
        validation_z, validation_y = predict(algorithm, validation_df)
        test_z, test_y = predict(algorithm, test_df)
        
        history.append({
            'k': k,
            'val_auc': evaluate_auc(validation_z, validation_y),
            'test_auc': evaluate_auc(test_z, test_y),
            'val_logloss': evaluate_logloss(validation_z, validation_y),
            'test_logloss': evaluate_logloss(test_z, test_y),
            'val_ndcg': evaluate_ndcg(validation_z, validation_y),
            'test_ndcg': evaluate_ndcg(test_z, test_y)
        })
    return history

In [23]:
history = train_ks(start=3, end=30, step=1)

Using K = 3
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 4
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 5
Computing the cosine similarity matrix...


...

Done computing similarity matrix.
Using K = 29
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 30
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.


In [24]:
print("| K | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |")
print("|:-- | -- | -- | -- | -- | -- | -- |")
for his in history:
    print("| k={} | {:.5f} | {:.5f} | {:.5f} | {:.5f} | {:.5f} | {:.5f} |".format(
        his['k'],
        his['val_auc'],
        his['val_logloss'],
        his['val_ndcg'],
        his['test_auc'],
        his['test_logloss'],
        his['test_ndcg'],
    ))

| K | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| k=3 | 0.89886 | 0.16942 | 0.99645 | 0.95404 | 0.09448 | 0.99840 |
| k=4 | 0.91065 | 0.16746 | 0.99648 | 0.96015 | 0.09351 | 0.99834 |
| k=5 | 0.92134 | 0.16633 | 0.99649 | 0.96459 | 0.09288 | 0.99835 |
| k=6 | 0.92506 | 0.16591 | 0.99649 | 0.96506 | 0.09507 | 0.99828 |
| k=7 | 0.92631 | 0.17175 | 0.99631 | 0.96707 | 0.09468 | 0.99828 |
| k=8 | 0.92737 | 0.17163 | 0.99631 | 0.96931 | 0.09221 | 0.99835 |
| k=9 | 0.92947 | 0.17136 | 0.99631 | 0.96979 | 0.09210 | 0.99836 |
| k=10 | 0.93046 | 0.17127 | 0.99631 | 0.97048 | 0.09207 | 0.99836 |
| k=11 | 0.93189 | 0.17117 | 0.99631 | 0.97086 | 0.09199 | 0.99836 |
| k=12 | 0.93373 | 0.17104 | 0.99631 | 0.97115 | 0.09198 | 0.99836 |
| k=13 | 0.93712 | 0.17076 | 0.99631 | 0.97129 | 0.09192 | 0.99836 |
| k=14 | 0.93870 | 0.17072 | 0.99631 | 0.97179 | 0.09192 | 0.99836 |
| k=15 | 0.93871 | 0.17063 | 

## Experiment Results

LON-A dataset:  

| settings | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| **k=3*** | 0.49558 | 2.35525 | 0.93030 | 0.51150 | 2.29329 | 0.93262 |
| k=4 | 0.48577 | 2.40694 | 0.92655 | 0.49634 | 2.38645 | 0.92540 |
| k=5 | 0.47602 | 2.43115 | 0.92370 | 0.48876 | 2.43786 | 0.92147 |
| k=6 | 0.46365 | 2.48746 | 0.91964 | 0.48159 | 2.47573 | 0.91847 |
| k=7 | 0.45771 | 2.51404 | 0.91761 | 0.47709 | 2.50212 | 0.91615 |
| k=8 | 0.45146 | 2.53207 | 0.91576 | 0.47356 | 2.50206 | 0.91463 |
| k=9 | 0.44490 | 2.54086 | 0.91427 | 0.46937 | 2.50618 | 0.91340 |
| k=10 | 0.44219 | 2.54377 | 0.91324 | 0.46620 | 2.51697 | 0.91228 |

NYC-R dataset:  

| settings | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| **k=3*** | 0.89886 | 0.16942 | 0.99645 | 0.95404 | 0.09448 | 0.99840 |
| k=4 | 0.91065 | 0.16746 | 0.99648 | 0.96015 | 0.09351 | 0.99834 |
| k=5 | 0.92134 | 0.16633 | 0.99649 | 0.96459 | 0.09288 | 0.99835 |
| k=6 | 0.92506 | 0.16591 | 0.99649 | 0.96506 | 0.09507 | 0.99828 |
| k=7 | 0.92631 | 0.17175 | 0.99631 | 0.96707 | 0.09468 | 0.99828 |
| k=8 | 0.92737 | 0.17163 | 0.99631 | 0.96931 | 0.09221 | 0.99835 |
| k=9 | 0.92947 | 0.17136 | 0.99631 | 0.96979 | 0.09210 | 0.99836 |
| k=10 | 0.93046 | 0.17127 | 0.99631 | 0.97048 | 0.09207 | 0.99836 |
| k=11 | 0.93189 | 0.17117 | 0.99631 | 0.97086 | 0.09199 | 0.99836 |
| k=12 | 0.93373 | 0.17104 | 0.99631 | 0.97115 | 0.09198 | 0.99836 |
| k=13 | 0.93712 | 0.17076 | 0.99631 | 0.97129 | 0.09192 | 0.99836 |
| k=14 | 0.93870 | 0.17072 | 0.99631 | 0.97179 | 0.09192 | 0.99836 |
| k=15 | 0.93871 | 0.17063 | 0.99631 | 0.97200 | 0.09178 | 0.99836 |
| k=16 | 0.93909 | 0.17056 | 0.99631 | 0.97240 | 0.09172 | 0.99836 |
| k=17 | 0.93909 | 0.17056 | 0.99631 | 0.97267 | 0.09169 | 0.99836 |
| k=18 | 0.93984 | 0.17047 | 0.99631 | 0.97286 | 0.09166 | 0.99836 |
| k=19 | 0.94020 | 0.17044 | 0.99631 | 0.97286 | 0.09166 | 0.99836 |
| k=20 | 0.94162 | 0.17032 | 0.99631 | 0.97317 | 0.09163 | 0.99836 |
| k=21 | 0.94265 | 0.17021 | 0.99631 | 0.97323 | 0.09162 | 0.99836 |
| k=22 | 0.94298 | 0.17015 | 0.99631 | 0.97330 | 0.09162 | 0.99836 |
| k=23 | 0.94331 | 0.17011 | 0.99631 | 0.97330 | 0.09162 | 0.99836 |
| k=24 | 0.94331 | 0.17011 | 0.99631 | 0.97330 | 0.09162 | 0.99836 |
| k=25 | 0.94363 | 0.17008 | 0.99631 | 0.97336 | 0.09161 | 0.99836 |
| k=26 | 0.94363 | 0.17008 | 0.99631 | 0.97336 | 0.09161 | 0.99836 |
| k=27 | 0.94361 | 0.17013 | 0.99631 | 0.97341 | 0.09164 | 0.99836 |
| k=28 | 0.94360 | 0.17013 | 0.99631 | 0.97341 | 0.09164 | 0.99836 |
| k=29 | 0.94361 | 0.17012 | 0.99631 | 0.97353 | 0.09162 | 0.99836 |
| **k=30*** | 0.94394 | 0.17005 | 0.99631 | 0.97359 | 0.09161 | 0.99836 |